<a href="https://colab.research.google.com/github/Laurahg22/Trabajo-analitica-RH/blob/main/e_despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd ### para manejo de datos
import sqlite3 as sql
import joblib
import openpyxl ## para exportar a excel
import numpy as np
import sys #para ver la ruta

sys.path ###Ruta directorio qué tiene paquetes
sys.path.append('/content/drive/MyDrive/trabajo/Trabajo-analitica-RH') ## este comanda agrega una ruta

import a_funciones as funciones  ###archivo de funciones propias

In [3]:
### funcion para cargar objeto guardado ###
rf_final = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/rf_final.pkl")
dt_final = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/dt_final.pkl")
list_cat=joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/var_cat.pkl")
list_dummies=joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/list_dummies.pkl")
var_names=joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/var_names.pkl")
scaler=joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/scaler.pkl")
df_2016_d =joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/df_dummies_2016.pkl")

In [6]:
list_dummies

Index(['BusinessTravel', 'Department', 'EducationField', 'JobRole',
       'MaritalStatus'],
      dtype='object')

In [4]:
# Lectura de datos
df_2016_d

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,...,1.0,6,1,0,0,3.0,4.0,2.0,3,3
1,31,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,Research Scientist,...,6.0,3,5,1,4,3.0,2.0,4.0,2,4
2,32,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,Sales Executive,...,5.0,2,5,0,3,2.0,2.0,1.0,3,3
3,38,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,Human Resources,...,13.0,5,8,7,5,4.0,4.0,3.0,2,3
4,32,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,Sales Executive,...,9.0,2,6,0,4,4.0,1.0,3.0,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,Travel_Rarely,Research & Development,5,4,Medical,4406,Female,1,Research Scientist,...,10.0,5,3,0,2,4.0,1.0,3.0,3,3
4406,29,Travel_Rarely,Research & Development,2,4,Medical,4407,Male,1,Laboratory Technician,...,10.0,2,3,0,2,4.0,4.0,3.0,2,3
4407,25,Travel_Rarely,Research & Development,25,2,Life Sciences,4408,Male,2,Sales Executive,...,5.0,4,4,1,2,1.0,3.0,3.0,3,4
4408,42,Travel_Rarely,Sales,18,2,Medical,4409,Male,1,Laboratory Technician,...,10.0,2,9,7,8,4.0,1.0,3.0,2,3


#Automatizar

In [5]:
###### el despliegue consiste en dejar todo el código listo para una ejecucion automática en el periodo definido:
###### en este caso se ejecutara el proceso de entrenamiento y prediccion anualmente.
if __name__=="__main__":
    ## llamar base de datos
    df_2016_d
    ####Otras transformaciones en python (imputación, dummies y seleccion de variables)
    df_t= funciones.preparar_datos(df_2016_d)
    ##Cargar modelo y predecir
    dt_final = joblib.load("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/dt_final.pkl")
    predicciones=dt_final.predict(df_t)
    pd_pred=pd.DataFrame(predicciones, columns=['Retiros'])

    ###Crear base con predicciones ####
    perf_pred=pd.concat([df_2016_d['EmployeeID'],df_t,pd_pred],axis=1)
    ####ver_predicciones_bajas ###
    emp_pred_bajo=perf_pred.sort_values(by=["Predicciones retiros"],ascending=True).head(10)
    emp_pred_bajo.set_index('EmployeeID', inplace=True)
    pred=emp_pred_bajo.T
    coeficientes=pd.DataFrame( np.append(dt_final.intercept_,dt_final.coef_) , columns=['coeficientes'])  ### agregar coeficientes

    pred.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/prediccion.xlsx")   #### exportar predicciones mas bajas y variables explicativas
    coeficientes.to_excel("/content/drive/MyDrive/trabajo/Trabajo-analitica-RH/salidas/coeficientes.xlsx") ### exportar coeficientes para analizar predicciones



KeyError: "None of [Index(['BusinessTravel', 'Department', 'EducationField', 'JobRole',\n       'MaritalStatus'],\n      dtype='object')] are in the [columns]"